# Система рекомендации книг. Основана на коллаборативной фильтрации
# Books Recommender system. Collaborative filtering based

- Dataset :- https://www.kaggle.com/ra4u12/bookrecommendation

## Импортируем необходимые пакеты / Importing necessary libraries

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

## Считываем данные в датафреймы / Reading dataframes

In [3]:
# Считываем первый датафрейм / Loading the 1st dataframe
books = pd.read_csv('data/BX-Books.csv', sep=";", on_bad_lines='skip', encoding='latin-1')
books.head()

C:\Users\Илья\AppData\Local\Temp\ipykernel_6132\2333814344.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('data/BX-Books.csv', sep=";", on_bad_lines='skip', encoding='latin-1')


ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/images/P/0002005018.0...   
2  http://images.amazon.com/images/P/0060973129.0...   
3  http://images.amazon.com/images/P/0374157065.0...   
4  http://images.amazon.com/images/P/0393045218.0...   

                                         Image-URL-M  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/images/P/0002005018.0...   
2  http://images.amazon.com/images/P/0060973129.0...   
3  http://images.amazon.com/images/P/0374157065.0...   
4  http://images.amazon.com/images/P/0393045218.0...   

                                         Image-URL-L  
0  http://images.amazon.com/images/P/0195153448.0...  
1  http://images.amazon.com/images/P/0002005018.0...  
2  http://images.amazon.com/images/P/0060973129.0...  
3  http://images.amazon.com/images/P/0374157065.0...  
4  http://images.amazon.com/images/P/0393045218.0...

In [4]:
# Считываем второй датафрейм / Loading the 2nd dataframe
users = pd.read_csv('data/BX-Users.csv', sep=";", on_bad_lines='skip', encoding='latin-1')
users.head()

User-ID                            Location   Age
0        1                  nyc, new york, usa   NaN
1        2           stockton, california, usa  18.0
2        3     moscow, yukon territory, russia   NaN
3        4           porto, v.n.gaia, portugal  17.0
4        5  farnborough, hants, united kingdom   NaN

In [5]:
# Считываем третий датафрейм / Loading the 3rd dataframe
ratings = pd.read_csv('data/BX-Book-Ratings.csv', sep=";", on_bad_lines='skip', encoding='latin-1')
ratings.head()

User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6

## EDA

In [112]:
# Посмотрим на размерности датафреймов / Let's look at the shape of the dataframes
print(books.shape, users.shape, ratings.shape, sep='\n')

(271360, 8)
(278858, 3)
(1149780, 3)


In [113]:
# Посмотрим внимательней на то, что внутри данных / Let's have a closer look at what is inside the data

In [114]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [98]:
# Посмотрим сколько уникальных пользователей есть в датафрейме / 
# Let's have a look at how many unique users there are in the ratings dataframe
ratings['User-ID'].value_counts()

User-ID
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [100]:
ratings['User-ID'].unique().shape

(105283,)

In [25]:
# Проверим наличие пропущенных значений
# Checking the number of missing values
books.isna().sum()

ISBN         0
title        0
author       2
year         0
publisher    2
image_url    3
dtype: int64

In [26]:
ratings.isna().sum()

user_id    0
ISBN       0
rating     0
dtype: int64

In [27]:
users.isna().sum()

user_id          0
location         0
age         110762
dtype: int64

In [28]:
# Датафрейм 'users' не представляет интереса для нашей задачи
# 'users' dataframe does not seem to be interesting for our objective

## Data Preparation

In [6]:
# Удалим некоторые ненужные колонки / Lets delete some columns
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']]
books.head()

ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-L  
0  http://images.amazon.com/images/P/0195153448.0...  
1  http://images.amazon.com/images/P/0002005018.0...  
2  http://images.amazon.com/images/P/0060973129.0...  
3  http://images.amazon.com/images/P/0374157065.0...  
4  http://images.amazon.com/images/P/0393045218.0...

In [7]:
# Для простоты переименуем некоторые колонки / Lets remane some columns name
books.rename(columns={"Book-Title": 'title',
                      'Book-Author': 'author',
                     "Year-Of-Publication": 'year',
                     "Publisher": "publisher",
                     "Image-URL-L": "image_url"}, inplace=True)

In [119]:
books.head()

ISBN                                              title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 author  year                   publisher  \
0    Mark P. O. Morford  2002     Oxford University Press   
1  Richard Bruce Wright  2001       HarperFlamingo Canada   
2          Carlo D'Este  1991             HarperPerennial   
3      Gina Bari Kolata  1999        Farrar Straus Giroux   
4       E. J. W. Barber  1999  W. W. Norton &amp; Company   

                                           image_url  
0  http://images.amazon.com/images/P/0195153448.0...  
1  http://images.amazon.com/images/P/0002005018.0...  
2  http://images.amazon.com/images/P/0060973129.0...  
3  http://images.amazon.com/images/P/0374157065.0...  
4  http://images.amazon.com/images/P/0393045218.0...

In [8]:
users.rename(columns={"User-ID":'user_id',
                      'Location':'location',
                     "Age":'age'},inplace=True)

In [121]:
users.head(2)

user_id                   location   age
0        1         nyc, new york, usa   NaN
1        2  stockton, california, usa  18.0

In [9]:
ratings.rename(columns={"User-ID":'user_id',
                      'Book-Rating':'rating'},inplace=True)

In [123]:
ratings.head(2)

user_id        ISBN  rating
0   276725  034545104X       0
1   276726  0155061224       5

In [10]:
# Сохраним пользователей, кто оценил более 200 книг
# Let's store users who have rated more than 200 books
users_200 = ratings['user_id'].value_counts() > 200

In [11]:
users_200

user_id
11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
116180    False
116166    False
116154    False
116137    False
276723    False
Name: count, Length: 105283, dtype: bool

In [136]:
users_200[users_200].shape

(899,)

In [137]:
users_200_index = users_200[users_200].index
users_200_index

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [139]:
# Теперь сохраним рейтинги, которые проставили данные пользователи
# Let's store ratings of those users
ratings = ratings[ratings['user_id'].isin(users_200_index)]
ratings.head()

user_id        ISBN  rating
1456   277427  002542730X      10
1457   277427  0026217457       0
1458   277427  003008685X       8
1459   277427  0030615321       0
1460   277427  0060002050       0

In [140]:
ratings.shape

(526356, 3)

In [141]:
# Теперь смерджим получившийся датафрейм с датафреймом книг
# Now we're joining ratings with books
ratings_with_books = ratings.merge(books, on='ISBN')

In [142]:
ratings_with_books.head()

user_id        ISBN  rating  \
0   277427  002542730X      10   
1     3363  002542730X       0   
2    11676  002542730X       6   
3    12538  002542730X      10   
4    13552  002542730X       0   

                                               title             author  year  \
0  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
1  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
2  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
3  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
4  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   

                   publisher  \
0  John Wiley &amp; Sons Inc   
1  John Wiley &amp; Sons Inc   
2  John Wiley &amp; Sons Inc   
3  John Wiley &amp; Sons Inc   
4  John Wiley &amp; Sons Inc   

                                           image_url  
0  http://images.amazon.com/images/P/002542730X.0...  
1  http://images.amazon.com/images/P/002542730X.0...  
2  http://images.amazon.com/images/P/002542730X.0...  
3  http://images.amazon.com/images/P/002542730X.0...  
4  http://images.amazon.com/images/P/002542730X.0...

In [143]:
ratings_with_books.shape

(487671, 8)

In [145]:
# Посчитаем сколько раз оценивалась каждая книга
# Let's count how many times each book has been rated
count_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()
count_rating.head()

title  rating
0   A Light in the Storm: The Civil War Diary of ...       2
1                              Always Have Popsicles       1
2               Apple Magic (The Collector's series)       1
3   Beyond IBM: Leadership Marketing and Finance ...       1
4   Clifford Visita El Hospital (Clifford El Gran...       1

In [146]:
# Переименуем некоторые колонки
# Let's rename some column names
count_rating.rename(columns={'rating': 'num_of_ratings'}, inplace=True)
count_rating.head()

title  num_of_ratings
0   A Light in the Storm: The Civil War Diary of ...               2
1                              Always Have Popsicles               1
2               Apple Magic (The Collector's series)               1
3   Beyond IBM: Leadership Marketing and Finance ...               1
4   Clifford Visita El Hospital (Clifford El Gran...               1

In [147]:
# Теперь смерджим получившийся датафрейм с датафреймом книги
# Now we're joining count_rating with books
final_rating = ratings_with_books.merge(count_rating, on='title')
final_rating.head()

user_id        ISBN  rating  \
0   277427  002542730X      10   
1     3363  002542730X       0   
2    11676  002542730X       6   
3    12538  002542730X      10   
4    13552  002542730X       0   

                                               title             author  year  \
0  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
1  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
2  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
3  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
4  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   

                   publisher  \
0  John Wiley &amp; Sons Inc   
1  John Wiley &amp; Sons Inc   
2  John Wiley &amp; Sons Inc   
3  John Wiley &amp; Sons Inc   
4  John Wiley &amp; Sons Inc   

                                           image_url  num_of_ratings  
0  http://images.amazon.com/images/P/002542730X.0...              82  
1  http://images.amazon.com/images/P/002542730X.0...              82  
2  http://images.amazon.com/images/P/002542730X.0...              82  
3  http://images.amazon.com/images/P/002542730X.0...              82  
4  http://images.amazon.com/images/P/002542730X.0...              82

In [148]:
final_rating.shape

(487671, 9)

In [150]:
# Возьмем только те книги, которые оценивались хотя бы 50 раз
# Let's take the books which got at least 50 ratings
final_rating = final_rating[final_rating['num_of_ratings'] >= 50]
final_rating.head()

user_id        ISBN  rating  \
0   277427  002542730X      10   
1     3363  002542730X       0   
2    11676  002542730X       6   
3    12538  002542730X      10   
4    13552  002542730X       0   

                                               title             author  year  \
0  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
1  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
2  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
3  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   
4  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  1994   

                   publisher  \
0  John Wiley &amp; Sons Inc   
1  John Wiley &amp; Sons Inc   
2  John Wiley &amp; Sons Inc   
3  John Wiley &amp; Sons Inc   
4  John Wiley &amp; Sons Inc   

                                           image_url  num_of_ratings  
0  http://images.amazon.com/images/P/002542730X.0...              82  
1  http://images.amazon.com/images/P/002542730X.0...              82  
2  http://images.amazon.com/images/P/002542730X.0...              82  
3  http://images.amazon.com/images/P/002542730X.0...              82  
4  http://images.amazon.com/images/P/002542730X.0...              82

In [151]:
final_rating.shape

(61853, 9)

In [152]:
# Удалим дубликаты
# let's drop the duplicates
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [153]:
final_rating.shape

(59850, 9)

In [155]:
# Создаем сводную таблицу
# Let's create a pivot table
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values='rating')
book_pivot

user_id                                             254     2276    2766    \
title                                                                        
1984                                                   9.0     NaN     NaN   
1st to Die: A Novel                                    NaN     NaN     NaN   
2nd Chance                                             NaN    10.0     NaN   
4 Blondes                                              NaN     NaN     NaN   
84 Charing Cross Road                                  NaN     NaN     NaN   
...                                                    ...     ...     ...   
Year of Wonders                                        NaN     NaN     NaN   
You Belong To Me                                       NaN     NaN     NaN   
Zen and the Art of Motorcycle Maintenance: An I...     NaN     NaN     NaN   
Zoya                                                   NaN     NaN     NaN   
\O\" Is for Outlaw"                                    NaN     NaN     NaN   

user_id                                             2977    3363    3757    \
title                                                                        
1984                                                   NaN     NaN     NaN   
1st to Die: A Novel                                    NaN     NaN     NaN   
2nd Chance                                             NaN     NaN     NaN   
4 Blondes                                              NaN     NaN     NaN   
84 Charing Cross Road                                  NaN     NaN     NaN   
...                                                    ...     ...     ...   
Year of Wonders                                        7.0     NaN     NaN   
You Belong To Me                                       NaN     NaN     NaN   
Zen and the Art of Motorcycle Maintenance: An I...     NaN     0.0     NaN   
Zoya                                                   NaN     NaN     NaN   
\O\" Is for Outlaw"                                    NaN     NaN     NaN   

user_id                                             4017    4385    6242    \
title                                                                        
1984                                                   NaN     NaN     NaN   
1st to Die: A Novel                                    NaN     NaN     NaN   
2nd Chance                                             NaN     NaN     NaN   
4 Blondes                                              NaN     NaN     NaN   
84 Charing Cross Road                                  NaN     NaN     NaN   
...                                                    ...     ...     ...   
Year of Wonders                                        NaN     NaN     7.0   
You Belong To Me                                       NaN     NaN     NaN   
Zen and the Art of Motorcycle Maintenance: An I...     NaN     NaN     NaN   
Zoya                                                   NaN     NaN     NaN   
\O\" Is for Outlaw"                                    NaN     NaN     NaN   

user_id                                             6251    ...  274004  \
title                                                       ...           
1984                                                   NaN  ...     NaN   
1st to Die: A Novel                                    NaN  ...     NaN   
2nd Chance                                             NaN  ...     NaN   
4 Blondes                                              0.0  ...     NaN   
84 Charing Cross Road                                  NaN  ...     NaN   
...                                                    ...  ...     ...   
Year of Wonders                                        NaN  ...     NaN   
You Belong To Me                                       NaN  ...     NaN   
Zen and the Art of Motorcycle Maintenance: An I...     0.0  ...     NaN   
Zoya                                                   NaN  ...     NaN   
\O\" Is for Outlaw"                                    

In [156]:
book_pivot.shape

(742, 888)

In [159]:
# Заполняем пустые значения нулями
# Filling NaN values with zeros
book_pivot.fillna(value=0, inplace=True)

In [160]:
book_pivot

user_id                                             254     2276    2766    \
title                                                                        
1984                                                   9.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0    10.0     0.0   
4 Blondes                                              0.0     0.0     0.0   
84 Charing Cross Road                                  0.0     0.0     0.0   
...                                                    ...     ...     ...   
Year of Wonders                                        0.0     0.0     0.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
Zoya                                                   0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0.0     0.0   

user_id                                             2977    3363    3757    \
title                                                                        
1984                                                   0.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0     0.0     0.0   
4 Blondes                                              0.0     0.0     0.0   
84 Charing Cross Road                                  0.0     0.0     0.0   
...                                                    ...     ...     ...   
Year of Wonders                                        7.0     0.0     0.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
Zoya                                                   0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0.0     0.0   

user_id                                             4017    4385    6242    \
title                                                                        
1984                                                   0.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
2nd Chance                                             0.0     0.0     0.0   
4 Blondes                                              0.0     0.0     0.0   
84 Charing Cross Road                                  0.0     0.0     0.0   
...                                                    ...     ...     ...   
Year of Wonders                                        0.0     0.0     7.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
Zoya                                                   0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0.0     0.0   

user_id                                             6251    ...  274004  \
title                                                       ...           
1984                                                   0.0  ...     0.0   
1st to Die: A Novel                                    0.0  ...     0.0   
2nd Chance                                             0.0  ...     0.0   
4 Blondes                                              0.0  ...     0.0   
84 Charing Cross Road                                  0.0  ...     0.0   
...                                                    ...  ...     ...   
Year of Wonders                                        0.0  ...     0.0   
You Belong To Me                                       0.0  ...     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0  ...     0.0   
Zoya                                                   0.0  ...     0.0   
\O\" Is for Outlaw"                                    

In [169]:
# Изменим тип данных в датасете
# Changing the data type in the dataset
book_pivot = book_pivot.astype(dtype='int16', copy=True)
book_pivot

user_id                                             254     2276    2766    \
title                                                                        
1984                                                     9       0       0   
1st to Die: A Novel                                      0       0       0   
2nd Chance                                               0      10       0   
4 Blondes                                                0       0       0   
84 Charing Cross Road                                    0       0       0   
...                                                    ...     ...     ...   
Year of Wonders                                          0       0       0   
You Belong To Me                                         0       0       0   
Zen and the Art of Motorcycle Maintenance: An I...       0       0       0   
Zoya                                                     0       0       0   
\O\" Is for Outlaw"                                      0       0       0   

user_id                                             2977    3363    3757    \
title                                                                        
1984                                                     0       0       0   
1st to Die: A Novel                                      0       0       0   
2nd Chance                                               0       0       0   
4 Blondes                                                0       0       0   
84 Charing Cross Road                                    0       0       0   
...                                                    ...     ...     ...   
Year of Wonders                                          7       0       0   
You Belong To Me                                         0       0       0   
Zen and the Art of Motorcycle Maintenance: An I...       0       0       0   
Zoya                                                     0       0       0   
\O\" Is for Outlaw"                                      0       0       0   

user_id                                             4017    4385    6242    \
title                                                                        
1984                                                     0       0       0   
1st to Die: A Novel                                      0       0       0   
2nd Chance                                               0       0       0   
4 Blondes                                                0       0       0   
84 Charing Cross Road                                    0       0       0   
...                                                    ...     ...     ...   
Year of Wonders                                          0       0       7   
You Belong To Me                                         0       0       0   
Zen and the Art of Motorcycle Maintenance: An I...       0       0       0   
Zoya                                                     0       0       0   
\O\" Is for Outlaw"                                      0       0       0   

user_id                                             6251    ...  274004  \
title                                                       ...           
1984                                                     0  ...       0   
1st to Die: A Novel                                      0  ...       0   
2nd Chance                                               0  ...       0   
4 Blondes                                                0  ...       0   
84 Charing Cross Road                                    0  ...       0   
...                                                    ...  ...     ...   
Year of Wonders                                          0  ...       0   
You Belong To Me                                         0  ...       0   
Zen and the Art of Motorcycle Maintenance: An I...       0  ...       0   
Zoya                                                     0  ...       0   
\O\" Is for Outlaw"                                    

In [170]:
book_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 742 entries, 1984 to \O\" Is for Outlaw"
Columns: 888 entries, 254 to 278418
dtypes: int16(888)
memory usage: 1.3+ MB


# Modeling

In [198]:
# Для обучения модели сохраним данные в формате numpy array
# To train the model we'll save the data as numpy array
book_numpy = book_pivot.to_numpy()
book_numpy

array([[ 9,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0, 10,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int16)

In [174]:
# Создаем экземпляр класса NearestNeighbors. Это модель машинного обучения без учителя
# Creating Nearest Neighbors instance. This is an unsupervised ML model
model = NearestNeighbors(algorithm='brute')

In [200]:
# Обучаем модель
# Training model
model.fit(book_numpy)

NearestNeighbors(algorithm='brute')

In [201]:
# Для получения рекомендаций от модели применим метод kneighbors()
# To elicit suggestions from the model kneighbors() method is used
suggestions = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6, return_distance=False)
suggestions

array([[237, 240, 238, 241, 184, 536]], dtype=int64)

In [204]:
# Напечатаем названия рекомендованных книг
# Let's print the titles of the suggested books
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [216]:
# Сохраним названия книг в отдельный список. Он понадобится для создания приложения
# Let's save the titles of the books to a list
book_names = book_pivot.index
book_names

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [217]:
book_names[2]

'2nd Chance'

In [197]:
np.where(book_pivot.index == '4 Blondes')[0][0]

3

#  Url extraction

In [218]:
# Для отображения картинок в приложении, необходимо извлечение ссылок на них
# To show the covers of the recommended books we need to extract the urls
book_id = np.where(final_rating['title'] == "Harry Potter and the Chamber of Secrets (Book 2)")[0][0]
book_id

4364

In [219]:
final_rating.iloc[book_id]['image_url']

'http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg'

In [220]:
book_name = []
for book_id in suggestion:
    book_name.append(book_pivot.index[book_id])  

In [221]:
book_name[0]

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')

In [222]:
ids_index = []
for name in book_name[0]: 
    ids = np.where(final_rating['title'] == name)[0][0]
    ids_index.append(ids)

In [223]:
for idx in ids_index:
    url = final_rating.iloc[idx]['image_url']
    print(url)

http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0439136369.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0439139597.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/043936213X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0446604232.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0440115450.01.LZZZZZZZ.jpg


In [225]:
# Сохраним обученную модель и необходимые датафреймы в файлы
# Saving the model and necessary data to files
pickle.dump(model, open('pickles/model.pkl','wb'))
pickle.dump(book_names, open('pickles/book_names.pkl','wb'))
pickle.dump(final_rating, open('pickles/final_rating.pkl','wb'))
pickle.dump(book_pivot, open('pickles/book_pivot.pkl','wb'))

# Testing model

In [79]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    suggestions = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6, return_distance=False)
    
    for i in range(len(suggestions)):
            books = book_pivot.index[suggestions[i]]
            for j in books:
                if j == book_name:
                    print(f"You searched '{book_name}'\n")
                    print("The suggestion books are: \n")
                else:
                    print(j)

In [228]:
book_name = "Harry Potter and the Chamber of Secrets (Book 2)"
recommend_book(book_name)

You searched 'Harry Potter and the Chamber of Secrets (Book 2)'

The suggestion books are: 

Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
The Cradle Will Fall
